# Parametric Model Document Formulas

This notebook attempts to generate the equations as shown in the parametric model document. Useful for checking whether the parametric model as implemented actually matches the document.

Last run with Jupyter Notebook 5.0.0 running Python 3.5.2

In [ ]:
import sys

from IPython.display import display, Math, Latex, HTML, Markdown
from sympy import latex, simplify, Lambda, Symbol

sys.path+=['..']
from sdp_par_model import evaluate
from sdp_par_model import reports as iapi
from sdp_par_model.config import PipelineConfig
from sdp_par_model.parameters.definitions import *
from sdp_par_model.parameters.container import BLDep

In [ ]:
# Telescope and band should not matter
cfg = PipelineConfig(telescope=Telescopes.SKA1_Mid, band=Bands.Mid1, pipeline=Pipelines.ICAL)
cfg_rcal = PipelineConfig(telescope=Telescopes.SKA1_Mid, band=Bands.Mid1, pipeline=Pipelines.RCAL)
cfg_ingest = PipelineConfig(telescope=Telescopes.SKA1_Mid, band=Bands.Mid1, pipeline=Pipelines.Ingest)
cfg_specfit = PipelineConfig(telescope=Telescopes.SKA1_Mid, band=Bands.Mid1, pipeline=Pipelines.DPrepA_Image)
tp = cfg.calc_tel_params(symbolify='product')
tp2 = cfg.calc_tel_params(symbolify='helper')
tp3 = cfg.calc_tel_params(symbolify='all')
tp_rcal = cfg_rcal.calc_tel_params(symbolify='product')
tp_ingest = cfg_ingest.calc_tel_params(symbolify='product')
tp_ingest2 = cfg_ingest.calc_tel_params(symbolify='helper')
tp_specfit = cfg_specfit.calc_tel_params(symbolify='product')
def show_sympy(*exprs):
    l = "=".join([latex(e) for e in exprs])
    display(Math("\\begin{aligned}" + l + "\\end{aligned}"))

In [ ]:
b = Symbol("b")
tp_ingest.Rviscorr = tp_ingest.Nbeam * tp_ingest.Npp * tp_ingest.Rvis_ingest
tp_ingest2.Rviscorr = tp_ingest2.Nbeam * tp_ingest2.Npp * tp_ingest2.Rvis_ingest
tp.RvisfullFoV = tp_ingest2.Nbeam * tp_ingest2.Npp * tp.Rvis_predict
# Definition of Rvis consistent with most uses in the document, and a helper to introduce it into formulas
RvisfullFoV = tp.Nbeam * tp.Nbl * tp.Npp * tp.Nf_vis / tp.Tint_used
def rvisfullfov(expr):
    return expr / RvisfullFoV(b) * Symbol("R_vis,fullFoV")

## Imaging

In [ ]:
# Ingest rate over all polarisations and beams.
show_sympy(Symbol("R_vis,corr"), tp_ingest.Rviscorr, tp_ingest2.Rviscorr)

In [ ]:
# Pixel production rate (after reprojection), assumed to be summed over the entire observation
show_sympy(Symbol("R_pix"), tp.products[Products.Reprojection]['Rout'] * tp.Tsnap / tp.Tobs)

## Receive

In [ ]:
show_sympy(Symbol("C_Receive"), tp_ingest.products[Products.Receive]['Rflop'])

## Flagging

In [ ]:
show_sympy(Symbol("N_flop/vis"), tp_ingest.products[Products.Flag]['Rflop'] / tp_ingest.Rviscorr)
show_sympy(Symbol("C_Flag"), tp_ingest.products[Products.Flag]['Rflop'])

## Demix

In [ ]:
show_sympy(Symbol("N_flop/vis"), tp_ingest.products[Products.Demix]['Rflop'] / tp_ingest.Rviscorr)
show_sympy(Symbol("C_Demix"), tp_ingest.products[Products.Demix]['Rflop'])

## Averaging

In [ ]:
show_sympy(Symbol("C_Average"), tp_ingest.products[Products.Average]['Rflop'])

## Predict via Direct Fourier Transform

In [ ]:
show_sympy(Symbol("N_flop/vis^predict,DFT"), tp.products[Products.DFT]['Rflop'] / RvisfullFoV(b) / tp.Nmajortotal)
show_sympy(Symbol("C_Predict"), rvisfullfov(tp.products[Products.DFT]['Rflop']))

## Solve

In [ ]:
show_sympy(Symbol("N_{flop/solution}^{StEFCal}"), tp.NFlop_solver)
show_sympy(Symbol("C_{Solve}^{RCAL}"), tp_rcal.products[Products.Solve]['Rflop'])
show_sympy(Symbol("C_{Solve}^{ICAL}"), tp.products[Products.Solve]['Rflop'])

## Subtract

In [ ]:
show_sympy(Symbol("C_Subtract"), tp.products[Products.Subtract_Visibility]['Rflop'])

## Correct

In [ ]:
show_sympy(Symbol("C_Correct"), tp.products[Products.Correct]['Rflop'])

## Phase Rotation

In [ ]:
show_sympy(Symbol("C_phrot,back"), simplify(tp.products[Products.PhaseRotation]['Rflop']), RvisfullFoV(b))
show_sympy(Symbol("C_phrot,predict"), rvisfullfov(tp.products[Products.PhaseRotationPredict]['Rflop']))

## Grid and Degrid

In [ ]:
show_sympy(Symbol("C_{grid}^{ICAL}"), tp.products[Products.Grid]['Rflop'])
show_sympy(Symbol("C_{degrid}^{ICAL,A}"), tp.products[Products.Degrid]['Rflop'])

## Gridding Kernel Update

In [ ]:
show_sympy(tp.Nkernel_AW_predict(b), tp2.Nkernel_AW_predict.term)
show_sympy(Symbol("N_cvff")(b), (tp.Ngcf_used_backward * tp.Nkernel_AW_predict).term)
show_sympy(Symbol("C_{Kernels,back}"), tp.products[Products.Gridding_Kernel_Update]['Rflop'])
show_sympy(Symbol("C_{Kernels,predict}"), tp.products[Products.Degridding_Kernel_Update]['Rflop'])

## FFT and iFFT

In [ ]:
show_sympy(tp.Nf_FFT_backward, tp3.Nf_FFT_backward)
show_sympy(Symbol("C_{FFT}^{ICAL}"), tp.products[Products.FFT]['Rflop'])
show_sympy(tp.Nf_FFT_predict, tp3.Nf_FFT_predict)
show_sympy(Symbol("C_{IFFT}^{ICAL,A}"), tp.products[Products.IFFT]['Rflop'])

## Reprojection

In [ ]:
show_sympy(Symbol("C_{Reproj}^{ICAL}"), tp.products[Products.Reprojection]['Rflop'])

## Image Spectral Fitting

In [ ]:
show_sympy(Symbol("C_{SpecFit}^{ICAL}"), tp_specfit.products[Products.Image_Spectral_Fitting]['Rflop'])

## Subtract Image Component

In [ ]:
show_sympy(Symbol("C_{grid}^{ICAL}"), tp.products[Products.Identify_Component]['Rflop'])
show_sympy(Symbol("C_{degrid}^{ICAL,A}"), tp.products[Products.Subtract_Image_Component]['Rflop'])

## Source find

In [ ]:
show_sympy(Symbol("C_{Source Find}"), tp.products[Products.Source_Find]['Rflop'])

# Time Smearing Limit, tsmear


In [ ]:
show_sympy(Symbol("t_{smear}")(b), tp2.Tint_used * tp2.combine_time_samples_facet.term)

# Number of Channels at the Frequency Smearing Limit, Nf,smear


In [ ]:
show_sympy(Symbol("Theta_PSF"), tp3.Theta_beam)
show_sympy(Symbol("Q_bw"), tp3.Qbw)
show_sympy(Symbol("N_f,smear")(b), tp2.Nf_no_smear_backward.term)

## Image Size

In [ ]:
from numpy import pi
# Replace wl_sb_max and pi by symbol in formula
show_sympy(Symbol("Theta_FoV"), tp3.Theta_fov_total /tp3.wl_sb_max*tp.wl_sb_max * pi/Symbol("pi"))

## Image Plane Pixel Size

In [ ]:
# Replace wl_sb_min by symbol in formula
show_sympy(Symbol("Theta_pix"), tp3.Theta_pix /tp3.wl_sb_min*tp.wl_sb_min)

## Number of Pixels on Image or Grid Size

In [ ]:
# All replacements from above
show_sympy(Symbol("N_pix"), tp3.Npix_linear_fov_total
            /tp3.wl_sb_max*tp.wl_sb_max *tp3.wl_sb_min/tp.wl_sb_min *pi/Symbol("pi"))

## Update Frequency Scale for Convolution Kernels

In [ ]:
show_sympy(Symbol("N_f,kernel")(b), tp2.Nf_gcf_backward.term)

## Size of the w-kernel

In [ ]:
show_sympy(1/(2*Symbol("pi")),1/2/pi)
show_sympy(Symbol("N_GW")(b), tp2.Ngw_backward.term)
show_sympy(tp.DeltaW_max.term, tp3.DeltaW_max.term)

## Imaging Pipeline Geometry Assumptions

In [ ]:
show_sympy(Symbol("\Delta w_min")(b), tp3.DeltaW_Earth.term)